# Melon 주간 Top 100 구하기

In [2]:
from bs4 import BeautifulSoup 
import urllib.request
import pandas as pd 

In [4]:
import requests
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers = header) 
html = req.text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

### 시작일 ~ 끝날

In [6]:
soup.select_one('.yyyymmdd').get_text()

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [7]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.11.09 ~ 2020.11.15'

In [8]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.11.09', '2020.11.15')

### 곡 리스트 찾기

In [10]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [16]:
rank = int(trs[0].select_one('.rank').string)
rank

1

In [34]:
for i in [0, 1, 2, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    print(diff)

0
1
-1
-100


In [39]:
info = trs[0].select_one('.ellipsis.rank01')
title = info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [41]:
info = trs[0].select('.ellipsis')
title = info[0].find('a').string
singer = info[1].find('a').string
album = info[2].find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

#### DataFrame으로 만들기

In [63]:
ranks = []; diffs = []
titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [64]:
for tr in trs:
    rank = int(tr.select_one('.rank').string)
    
    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string

    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    
    ranks.append(rank)
    diffs.append(diff)
    titles.append(title)
    singers.append(singer)
    albums.append(album)

In [68]:
top100 = pd.DataFrame({
            'rank': ranks,
            'diff': diffs,
            'title': titles,
            'singer': singers,
            'album': albums
})
top100 = top100.set_index('rank')
print(f'기간: {start_day} ~ {end_day}')
top100.head()

기간: 2020.11.09 ~ 2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,-1,Lovesick Girls,BLACKPINK,THE ALBUM
4,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,-1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME
